In [1]:
import os
import re
import nltk
import pandas as pd
import numpy as np

In [2]:
def pandas_config():
    pd.options.mode.chained_assignment = None
    pd.set_option('display.max_columns', None)
    # pd.set_option('display.max_rows', None)
    pd.set_option('display.expand_frame_repr', False)
    pd.set_option('max_colwidth', None)

pandas_config()

In [12]:
#sve recenice prisutne u df_cluster su labelirane sa 1
#ovo je izvuceno nakon klasifikovanja recenica u prvoj fazi
df = pd.read_excel('df_cluster.xlsx',  engine='openpyxl')

In [13]:
df.tail()

,Unnamed: 0,sentence,label,filename
439,271,"činjenica je da je oštećena zadobila povredu, što se konstatuje iz med.",NaN,data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10162.txt
440,272,"dokumentacije, međutim, oštećena tvrdi da joj povredu nije nanio okrivljeni, već je nastala od udara od ivicu vrata, kada je htjela da izađe iz prostorije.",NaN,data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10162.txt
441,273,pojasnila je da je iz revolta slučaj prijavila policiji i da su je braća nagovorila na to.,NaN,data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10162.txt
442,274,"pri ovako utvrđenom činjeničnom stanju, sud nije mogao izvesti zaključak, da je dokazano da je okrivljeni a. a. izvršio krivično djelo koje mu se optužnim predlogom stavlja na teret, odnosno da je primjenom grubog nasilja narušio tjelesni i duševni integritet člana svoje porodice, na način što je oštećenu, svoju suprugu, stisnutom pesnicom udario u predjelu liglaveca, već bi to bila predpostavka, na kojoj se sudska odluka ne može zasnovati.",NaN,data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10162.txt
443,275,"kako teza optužbe nije dokazana, sud je okrivljenog, na osnovu čl 363 tač 3 zkp-a, oslobodio od optužbe.",NaN,data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10162.txt


In [14]:
df.filename.value_counts()

data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10152.txt    91
data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10159.txt    56
data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10160.txt    54
data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10156.txt    53
data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10155.txt    50
data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10153.txt    44
data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10158.txt    43
data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10162.txt    32
data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10154.txt    21
Name: filename, dtype: int64

In [16]:
def func1(x):
    tokens1 = x.split(',')
    lst = []
    for t1 in tokens1:
        t1 = t1.rstrip()
        t1 = t1.strip()
        if len(t1.split(' '))>2:
            lst.append(t1)
    print(lst)
    print('-------\n')
    return lst
            
df.sentence = df.sentence.apply(lambda x: func1(x))

['b.“ doo m. b.', 'a u odsustvu ovlašćenog predstavnika oštećene fabrike „k.“', 'donio je i javno objavio dana 29.06.2011. godine', 'p r e s u d u i prema optuženima: 1. a. š.', 'od oca dž.']
-------

['i majke d.', 'rođenom … godine u k.', 'sa prebivalištem u naselju r.', 'ocu troje djece', 'školu nije učio', 'srednjeg imovnog stanja', 'neosuđivanom i 2. b. h.', 'od oca r. i majke lj.', 'rođenom … godine u p.', 'sa prebivalištem u naselju r.', 'školu nije učio', 'srednjeg imovnog stanja', 'neosuđivanom odbija se optužba da su: dana 07.12.2010. godine', 'oko 22:10 časova', 'u naselju r. u b.', 'po prethodnom dogovoru', 'zajedno obili pomoćni objekat fabrike "k."', 'na način što su upotrebom fizičke snage', 'razvalili vrata pomoćnog objekta fabrike', 'ušli u unutrašnjost istog i pokušali da oduzmu metalne djelove', 'u namjeri da njihovim prisvajanjem sebi pribave protiv pravnu imovinsku korist', 'u čemu su spriječeni od strane šefa obezbedjenja ž. v.', '- čime bi izvršili krivično djelo

In [18]:
df.sentence.values[-1]

['kako teza optužbe nije dokazana',
 'sud je okrivljenog',
 'na osnovu čl 363 tač 3 zkp-a',
 'oslobodio od optužbe.']

In [20]:
df.drop('label', inplace=True, axis=1)

KeyError: "['label'] not found in axis"

In [25]:
df.drop('Unnamed: 0', inplace=True, axis=1)

In [26]:
df

,sentence,filename
0,"[b.“ doo m. b., a u odsustvu ovlašćenog predstavnika oštećene fabrike „k.“, donio je i javno objavio dana 29.06.2011. godine, p r e s u d u i prema optuženima: 1. a. š., od oca dž.]",data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10152.txt
1,"[i majke d., rođenom … godine u k., sa prebivalištem u naselju r., ocu troje djece, školu nije učio, srednjeg imovnog stanja, neosuđivanom i 2. b. h., od oca r. i majke lj., rođenom … godine u p., sa prebivalištem u naselju r., školu nije učio, srednjeg imovnog stanja, neosuđivanom odbija se optužba da su: dana 07.12.2010. godine, oko 22:10 časova, u naselju r. u b., po prethodnom dogovoru, zajedno obili pomoćni objekat fabrike ""k."", na način što su upotrebom fizičke snage, razvalili vrata pomoćnog objekta fabrike, ušli u unutrašnjost istog i pokušali da oduzmu metalne djelove, u namjeri da njihovim prisvajanjem sebi pribave protiv pravnu imovinsku korist, u čemu su spriječeni od strane šefa obezbedjenja ž. v., - čime bi izvršili krivično djelo teška kradja u pokušaju iz čl 240 st. 1 tač 1 krivičnog zakonika, u vezi sa čl 20 krivičnog zakonika.]",data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10152.txt
2,"[usled odustanka odt-a berane od optužbe, na osnovu čl 362 tač 1 zkp-a.]",data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10152.txt
3,"[ii optuženi a. š. i b. h., oslobadjaju se od optužbe da su: u isto vrijeme i na istom mjestu, kao pod stavom i izreke presude, po prethodnom dogovoru, zajedno obili pomoćni objekat fabrike ""b.]",data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10152.txt
4,"[na način što su sa prozora pomoćnog objekta fabrike ""b.]",data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10152.txt
...,...,...
439,"[činjenica je da je oštećena zadobila povredu, što se konstatuje iz med.]",data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10162.txt
440,"[oštećena tvrdi da joj povredu nije nanio okrivljeni, već je nastala od udara od ivicu vrata, kada je htjela da izađe iz prostorije.]",data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10162.txt
441,[pojasnila je da je iz revolta slučaj prijavila policiji i da su je braća nagovorila na to.],data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10162.txt
442,"[pri ovako utvrđenom činjeničnom stanju, sud nije mogao izvesti zaključak, da je dokazano da je okrivljeni a. a. izvršio krivično djelo koje mu se optužnim predlogom stavlja na teret, odnosno da je primjenom grubog nasilja narušio tjelesni i duševni integritet člana svoje porodice, na način što je oštećenu, stisnutom pesnicom udario u predjelu liglaveca, već bi to bila predpostavka, na kojoj se sudska odluka ne može zasnovati.]",data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10162.txt


In [27]:
df3 = pd.DataFrame({col:np.repeat(df2['filename'].values, df2['sentence'].str.len())
         for col in df2.columns.difference(['sentence'])
        }).assign(**{'sentence':np.concatenate(df2['sentence'].values)})[df2.columns.tolist()]

In [29]:
df3[df3.filename == 'data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10154.txt']

,sentence,filename
520,osnovni sud u beranama k.br87/11 u ime naroda osnovni sud u beranama,data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10154.txt
521,u vijeću sastavljenom od sudije dubravke popović,data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10154.txt
522,kao predsjednika vijeća i sudija porotnika stojana kukalja i vučića babovića kao članova vijeća,data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10154.txt
523,uz sudjelovanje slavice janković,data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10154.txt
524,u krivičnom predmetu protiv okrivljenog t. m.,data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10154.txt
...,...,...
619,sud je uvjerenja da će se i uslovnom osudom kao mjerom upozorenja,data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10154.txt
620,uz prijetnju kaznom,data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10154.txt
621,u okviru opšte svrhe izricanja krivičnih sankcija predvidjenih čl4.st.2.kzcg postići svrha uslovne osude iz čl52.istog zakona i da će ovako odmjerena uslovna osuda predstavljati dovoljnu opomenu za okrivljenog i na njega uticati da ubuduće ne vrši krivična djela.,data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10154.txt
622,okrivljeni se obavezuje da na ime pričinjene štete plati oštećenoj pošti crne gore- pošti u b. iznos od 3.187.91 eura,data\K - PRVOSTEPENI KRIVIČNI PREDMETI\presuda_text_10154.txt


In [30]:
df3.to_csv('split_by_zarez_df_cluster.csv')